# Adding resveratrol-producing pathways

In [97]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

In [98]:
model_resv_glc = read_sbml_model('../data/models/iMM904.xml')

## Model 1: Resveratrol from glucose 

In [99]:
## Add reactions
# Reaction 1 (HA-TAL):L-tyrosine <=> (E).4.coumarate +   NH4+
new_reaction1 = Reaction('HA-TAL') # 3-(o)xopropanoate:(N)ADP+ (o)xidoreductase
coumarate4 = Metabolite(id='coumarate4', compartment='c')
new_reaction1.add_metabolites({model_resv_glc.metabolites.tyr__L_c: -1, 
                               coumarate4: 1, # 3-Oxopropanoate
                               model_resv_glc.metabolites.nh4_c: 1, # Search fpr NH4 ID
                              })
model_resv_glc.add_reactions([new_reaction1])

In [100]:
model_resv_glc.metabolites.coumarate4

Metabolite identifier,coumarate4
Name,
Memory address,0x12c02c7f0
Formula,None
Compartment,c
In 1 reaction(s),HA-TAL


In [101]:
new_reaction2 = Reaction('AT-4CL1')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction2.add_metabolites({model_resv_glc.metabolites.coumarate4: -1,
                               model_resv_glc.metabolites.atp_c: -1,
                               model_resv_glc.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_resv_glc.metabolites.amp_c: 1,
                               model_resv_glc.metabolites.ppi_c: 1,
                              })
model_resv_glc.add_reactions([new_reaction2])

In [102]:
new_reaction3 = Reaction('VVVST1')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction3.add_metabolites({model_resv_glc.metabolites.e4_coumaroyl_coa: -1,
                               model_resv_glc.metabolites.h_c: -3,
                               model_resv_glc.metabolites.malcoa_c: -3,
                               model_resv_glc.metabolites.co2_c: 4,
                               model_resv_glc.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_resv_glc.add_reactions([new_reaction3])

In [103]:
model_resv_glc.reactions.get_by_id('VVVST1')

Reaction identifier,VVVST1
Name,
Memory address,0x138e25750
Stoichiometry,e4_coumaroyl_coa + 3 h_c + 3 malcoa_c --> 4 co2_c + 4 coa_c + trans_resv + 3 H+ + 3 Malonyl CoA C24H33N7O19P3S --> 4 CO2 CO2 + 4 Coenzyme A +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [104]:
from cobra.io import write_sbml_model
write_sbml_model(model_resv_glc,'../data/models/yeast_resv_glc.xml')

from cobra.io import write_sbml_model
write_sbml_model(model,'data/models/yeast_resv_glc.xml')

In [105]:
model_resv_glc.add_boundary(model_resv_glc.metabolites.trans_resv, type='demand')

Reaction identifier,DM_trans_resv
Name,demand
Memory address,0x138ecc400
Stoichiometry,trans_resv --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [106]:
with model_resv_glc:
    model_resv_glc.objective=model_resv_glc.reactions.VVVST1
    lets_produce_resv= model_resv_glc.optimize().objective_value
lets_produce_resv

1.178571428571429

## Model 2: Resveratrol from coumaric acid from https://doi.org/10.1128/AEM.00609-06

In [107]:
model_resv_cou = read_sbml_model('../data/models/iMM904.xml')

In [108]:
new_reaction2_1 = Reaction('AT-4CL3')
coumarate4 = Metabolite(id='coumarate4', compartment='c')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction2_1.add_metabolites({coumarate4: -1,
                               model_resv_cou.metabolites.atp_c: -1,
                               model_resv_cou.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_resv_cou.metabolites.amp_c: 1,
                               model_resv_cou.metabolites.ppi_c: 1,
                              })

model_resv_cou.add_reactions([new_reaction2_1])

In [109]:
model_resv_cou.metabolites.coumarate4

Metabolite identifier,coumarate4
Name,
Memory address,0x12c253910
Formula,None
Compartment,c
In 1 reaction(s),AT-4CL3


In [110]:
new_reaction2_2 = Reaction('STS')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction2_2.add_metabolites({model_resv_cou.metabolites.e4_coumaroyl_coa: -1,
                               model_resv_cou.metabolites.h_c: -3,
                               model_resv_cou.metabolites.malcoa_c: -3,
                               model_resv_cou.metabolites.co2_c: 4,
                               model_resv_cou.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_resv_cou.add_reactions([new_reaction2_2])

In [111]:
from cobra.io import write_sbml_model
write_sbml_model(model_resv_cou,'../data/models/yeast_resv_cou.xml')

In [112]:
#how much resveratrol is produced?
model_resv_cou.add_boundary(model_resv_cou.metabolites.trans_resv, type='demand')

Reaction identifier,DM_trans_resv
Name,demand
Memory address,0x12c571210
Stoichiometry,trans_resv --> -->
GPR,
Lower bound,0
Upper bound,1000.0


## Model 3: Resveratrol from Phenylalanine (https://doi.org/10.1016/j.ymben.2009.07.004)


In [113]:
model_resv_phen = read_sbml_model('../data/models/iMM904.xml')

In [114]:
new_reaction3_1 = Reaction('PAL3')
e_cinnamate_c = Metabolite(id='e_cinnamate_c', compartment='c')
new_reaction3_1.add_metabolites({model_resv_phen.metabolites.phe__L_c: -1,
                               e_cinnamate_c: 1,
                               model_resv_phen.metabolites.nh4_c: 1,
                              })
model_resv_phen.add_reactions([new_reaction3_1])

In [115]:
new_reaction3_2 = Reaction('C4H') 
coumarate4 = Metabolite(id='coumarate4', compartment='c')
new_reaction3_2.add_metabolites({model_resv_phen.metabolites.e_cinnamate_c: -1,
                               model_resv_phen.metabolites.fmnh2_c: -1,
                               model_resv_phen.metabolites.o2_c: -1,
                               coumarate4: 1, 
                               model_resv_phen.metabolites.h2o_c: 1,
                               model_resv_phen.metabolites.h_c: 1,
                               model_resv_phen.metabolites.fmn_c: 1,
                              })
model_resv_phen.add_reactions([new_reaction3_2])

In [116]:
new_reaction3_3 = Reaction('AT-4CL2')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction3_3.add_metabolites({model_resv_phen.metabolites.coumarate4: -1,
                               model_resv_phen.metabolites.atp_c: -1,
                               model_resv_phen.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_resv_phen.metabolites.amp_c: 1,
                               model_resv_phen.metabolites.ppi_c: 1,
                              })
model_resv_phen.add_reactions([new_reaction3_3])

In [117]:
new_reaction3_4 = Reaction('STS')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction3_4.add_metabolites({model_resv_phen.metabolites.e4_coumaroyl_coa: -1,
                               model_resv_phen.metabolites.h_c: -3,
                               model_resv_phen.metabolites.malcoa_c: -3,
                               model_resv_phen.metabolites.co2_c: 4,
                               model_resv_phen.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_resv_phen.add_reactions([new_reaction3_4])

In [118]:
from cobra.io import write_sbml_model
write_sbml_model(model_resv_phen,'../data/models/yeast_resv_phen.xml')

In [119]:
model_resv_phen.add_boundary(model_resv_phen.metabolites.trans_resv, type='demand')

Reaction identifier,DM_trans_resv
Name,demand
Memory address,0x13a1179a0
Stoichiometry,trans_resv --> -->
GPR,
Lower bound,0
Upper bound,1000.0


### How much resveratrol is porduced?

In [120]:
# Model using glucose as subsrate
with model_resv_glc:
    model_resv_glc.objective=model_resv_glc.reactions.VVVST1
    lets_produce_resv= model_resv_glc.optimize().objective_value
lets_produce_resv

1.178571428571429

In [121]:
# Model using phenylananine
with model_resv_phen:
    model_resv_phen.objective=model_resv_phen.reactions.STS
    lets_produce_resv= model_resv_phen.optimize().objective_value
lets_produce_resv

1.0749185667752432

In [122]:
# model usin coumaric acid as substrate
with model_resv_cou:
    model_resv_cou.objective=model_resv_cou.reactions.STS
    lets_produce_resv= model_resv_cou.optimize().objective_value
lets_produce_resv

-3.389454704016765e-33